# Script - 3

Topics discussed in this script -
1. Nested & Repeated Data
2. Writing Efficient Queries

In [ ]:
%load_ext google.cloud.bigquery

Mounting the drive -

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Loading the service account key -

In [ ]:
link = 'STORE YOUR KEY IN A DRIVE AND WRITE PASTE SHAREABLE PATH HERE HERE'
 
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('scaler-ds-ml-de-75b4e150fe12.json')

Setting the environment variable -

In [ ]:
import os
cred_path = "scaler-ds-ml-de-75b4e150fe12.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=cred_path

In [ ]:
from google.cloud import bigquery

# Create a "Client" object (One time process)
client = bigquery.Client()

---
### Nested & Repeated Data
---

**We're using the 'ga_sessions_20170801' table from the 'google_analytics_sample' dataset.**

**Problem Statement -**


The team at google analytics is currently working on a research to study and understand their user's behavior.

They wish to find out which web browsers are most preferably used by the users and what pages of the website do they visit the most.

For this, the team has sampled out a small set of user interaction data collected during the GA sessions on 1st August 2017.

During their analysis, the team encounters an issue. The data available in some of the columns seems to be nested, repeated or both. 

Your job is to help them in resolving this problem and fetch the desired results.


**'ga_sessions_20170801' table -**

**Note**: Please refer to this [link](https://support.google.com/analytics/answer/3437719?hl=en) for data description.

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "google_analytics_sample" dataset
dataset_ref = client.dataset("google_analytics_sample", project="bigquery-public-data")

# Construct a reference to the "ga_sessions_20170801" table
table_ref = dataset_ref.table("ga_sessions_20170801")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1501650420,1501650420,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu...",1345240492995286662,None,None,Organic Search,Not Socially Engaged
1,<NA>,27,1501656919,1501656919,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",3884810646891698298,None,None,Direct,Not Socially Engaged
2,<NA>,1,1501596340,1501596340,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut...",1580431283728289185,None,None,Direct,Not Socially Engaged
3,<NA>,25,1501584669,1501584669,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",3884810646891698298,None,None,Direct,Not Socially Engaged
4,<NA>,26,1501608753,1501608753,20170801,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 10, 'minu...",3884810646891698298,None,None,Direct,Not Socially Engaged


But first, let's try to understand the structure of the data that we have.

**Nested data** -

<img src='https://drive.google.com/uc?id=1mbsff2IbGdqc-g7QO9uXWjJhNNuq1ktd'>

Nested columns have type STRUCT (or type RECORD) which is reflected in the table schema.

**How to access Nested columns -**

To query a column with nested data, we need to identify each field in the context of the column that contains it.
- Toy.Name refers to the "Name" field in the "Toy" column
- Toy.Type refers to the "Type" field in the "Toy" column

<img src='https://drive.google.com/uc?id=1wQG8uCnIxs6avDMaeJOrpQmi26whNUoT'>

   

**Repeated data** -

<img src='https://drive.google.com/uc?id=1hyWyyO5i-0AzeGIfqIkZ-nwKrguyr_pU'>

Repeated columns have mode REPEATED which is reflected in the table schema.

**How to access to repeated columns -**

- When querying repeated data, we need to put the name of the column containing the repeated data inside an UNNEST() function.
<img src='https://drive.google.com/uc?id=1Uzbv32Y-r9LqA_scIkMegWrVZbDk3Js8'>

- This essentially flattens the repeated data (which is then appended to the right side of the table) so that we have one element on each row. For an illustration of this, check out the image below.
<img src='https://drive.google.com/uc?id=1zo_QfHV8u5OQKQEo5uHnZon8n59vvpgc'>



Nested & Repeated data -

<img src='https://drive.google.com/uc?id=1nEQef63Vxmi-zxf-r6P_bATZZ7eBq-lV'>

**In the right table above we have column 'Toys' which is both repeated and nested. So how to query such column?**

- Let's look at a sample query
<img src='https://drive.google.com/uc?id=1kpr3WjB4w19j5WiSOzVg2gddskF_lzYl'>

- Since the "Toys" column is repeated, we flatten it with the UNNEST() function. And, since we give the flattened column an alias of t, we can refer to the "Name" and "Type" fields in the "Toys" column as t.Name and t.Type, respectively.


Q1. In order to find out the most popularly used web browser among the users during the GA session on 2017-08-01, we can use the total number of transactions done from each browser and rank them accordingly.



In [ ]:
query = """
        SELECT device.browser AS device_browser,       
            SUM(totals.transactions) as total_transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
        GROUP BY device_browser
        ORDER BY total_transactions DESC
        """

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()

,device_browser,total_transactions
0,Chrome,41
1,Safari,3
2,Firefox,1
3,Edge,<NA>
4,Coc Coc,<NA>


Q2. To determine the most popular landing point on their website, we can use the number of hits on each web page and order them accordingly.

In [ ]:
query = """
        SELECT hits.page.pagePath as path,
            COUNT(hits.page.pagePath) as counts
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`, 
            UNNEST(hits) as hits
        WHERE hits.type="PAGE" and hits.hitNumber=1
        GROUP BY path
        ORDER BY counts DESC
        """

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()

,path,counts
0,/home,1257
1,/google+redesign/shop+by+brand/youtube,587
2,/google+redesign/apparel/mens/mens+t+shirts,117
3,/signin.html,78
4,/basket.html,35


**How do we expand a nested column into multiple seperate columns?**

In [ ]:
expansion_query = """
                  SELECT 
                    visitId, visitorId, date,
                    totals.*
                  FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
                  """

# Run the query, and return a pandas DataFrame
result = client.query(expansion_query).result().to_dataframe()
result.head()

,visitId,visitorId,date,visits,hits,pageviews,timeOnSite,bounces,transactions,transactionRevenue,newVisits,screenviews,uniqueScreenviews,timeOnScreen,totalTransactionRevenue,sessionQualityDim
0,1501650420,<NA>,20170801,1,1,1,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1
1,1501656919,<NA>,20170801,1,1,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
2,1501596340,<NA>,20170801,1,1,1,<NA>,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,1
3,1501584669,<NA>,20170801,1,1,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
4,1501608753,<NA>,20170801,1,1,1,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1


---
### Writing Efficient Queries
---

**In this section of the lecture, we’ll be using the ‘bikeshare’ tables from the ‘san_francisco’ dataset.**

**Problem Statement -** 

Austin Bikes is a company that provides affordable & shared bike trips in the San Francisco Bay Area to some nearby cities.

They've recently made their data public and now wish to analyze this data in order to understand the customer demand in different regions so that they can figure out logistics for restocking bikes at each station.

Help them with this task using SQL. 

The three important factors that you should be looking at are:
* Most popular bike trips (start & end stations). 
* Number of bikes & docks available at different stations.
* When were these stations were installed.

**'bikeshare_trips' table -**

<img src='https://drive.google.com/uc?id=16u9pud1dI3_SrriugmhrE5w8wSpCNC3J'>

In [ ]:
# Construct a reference to the "san_francisco" dataset
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_trips" table
table_ref = dataset_ref.table("bikeshare_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


**bikeshare_stations' table -**

<img src='https://drive.google.com/uc?id=1wUeIA1nloJM-FaanZhPpAwtx3IcALK3D'>


In [ ]:
# Construct a reference to the "san_francisco" dataset
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_stations" table
table_ref = dataset_ref.table("bikeshare_stations")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,station_id,name,latitude,longitude,dockcount,landmark,installation_date
0,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,2013-08-06
1,84,Ryland Park,37.342725,-121.895617,15,San Jose,2014-04-09
2,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,2013-08-05
3,9,Japantown,37.348742,-121.894715,15,San Jose,2013-08-05
4,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,2013-08-05


**bikeshare_status' table -**

<img src='https://drive.google.com/uc?id=1H9UP205yKAo-HDslO-VwtFsE--Iy3dOY'>

In [ ]:
# Construct a reference to the "san_francisco" dataset
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_status" table
table_ref = dataset_ref.table("bikeshare_status")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,station_id,bikes_available,docks_available,time
0,2,0,27,2016-05-03 14:44:01+00:00
1,2,0,26,2014-10-06 22:30:03+00:00
2,2,0,27,2015-11-20 15:08:03+00:00
3,2,0,26,2014-10-06 22:17:02+00:00
4,2,0,27,2015-09-23 13:12:02+00:00


**Schema -**

<img src='https://drive.google.com/uc?id=11YW1OBJWDkmxyAOquL2tuP4gQ-WuE0L-'>

**We will use two functions to compare the efficiency of different queries:**

* show_amount_of_data_scanned() shows the amount of data the query uses.
* show_time_to_run() prints how long it takes for the query to execute.

In [ ]:
from google.cloud import bigquery
from time import time

client = bigquery.Client()

def show_amount_of_data_scanned(query):
    # dry_run lets us see how much data the query uses without running it
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    
    query_job = client.query(query, job_config=dry_run_config)
    print('Data processed: {} GB'.format(round(query_job.total_bytes_processed / 10**9, 3)))
    
def show_time_to_run(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    query_result = client.query(query, job_config=time_config).result()
    end = time()
    print('Time to run: {} seconds'.format(round(end-start, 3)))

#### **Objective 1. -** 
Just to have a clear look at the data in hand, fetch all the necessary details of bike trips present in the dataset provided.

**How would you do that?**

In [ ]:
star_query = """
              SELECT * 
              FROM `bigquery-public-data.san_francisco.bikeshare_trips`
              """

In [ ]:
show_amount_of_data_scanned(star_query)

Data processed: 0.126 GB


In [ ]:
show_time_to_run(star_query)

Time to run: 6.272 seconds


**How can we improve this?**

#### **Only select the columns you want.**

In [ ]:
basic_query = """
              SELECT trip_id, start_station_name, end_station_name 
              FROM `bigquery-public-data.san_francisco.bikeshare_trips`
              """

In [ ]:
show_amount_of_data_scanned(basic_query)

Data processed: 0.06 GB


In [ ]:
show_time_to_run(basic_query)

Time to run: 2.771 seconds


**Why does this happens?**

It is tempting to start queries with SELECT * FROM … It's convenient because you don't need to think about which columns you need. But it can be very inefficient. 

So try to fetch only the columns that you really need.


#### **Note:**

1. `COUNT(*)`- When * is used as an argument, it simply counts the total number of rows including the NULLs.
2. `COUNT(1)`- With COUNT(1), there is a misconception that it counts records from the first column. What COUNT(1) really does is that it replaces all the records you get from query result with the value 1 and then counts the rows meaning it even replaces a NULL with 1 meaning it takes NULLs into consideration while counting.
3. `COUNT(column_name)`- When a column name is used as an argument, it simply counts the total number of rows excluding the NULLs meaning it will not take NULLs into consideration.

In [ ]:
star_query = "SELECT COUNT(*) FROM `bigquery-public-data.san_francisco.bikeshare_trips`"
show_time_to_run(star_query)

basic_query = "SELECT COUNT(1) FROM `bigquery-public-data.san_francisco.bikeshare_trips`"
show_time_to_run(basic_query)

# No difference in performance

Time to run: 0.755 seconds
Time to run: 0.657 seconds


#### **Objective 2. -** 
Just to get an idea of how long do the bike trips usually are, write a SQL query to calculate the average duration of one-way bike trips in any one of the cities (let's say San Francisco).

**How would you do that?**

In [ ]:
more_data_query = """
                  SELECT MIN(start_station_name) AS start_station_name,
                      MIN(end_station_name) AS end_station_name,
                      AVG(duration_sec) AS avg_duration_sec
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE start_station_id != end_station_id 
                  GROUP BY start_station_id, end_station_id
                  LIMIT 10
                  """
                  
show_amount_of_data_scanned(more_data_query)

Data processed: 0.076 GB


**How can we improve this?**

#### **Read less data.**

In [ ]:
less_data_query = """
                  SELECT start_station_name,
                      end_station_name,
                      AVG(duration_sec) AS avg_duration_sec                  
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE start_station_name != end_station_name
                  GROUP BY start_station_name, end_station_name
                  LIMIT 10
                  """
                  
show_amount_of_data_scanned(less_data_query)

Data processed: 0.06 GB


**Why does this happens?**

Since there is a 1:1 relationship between the station ID and the station name, we don't need to use the start_station_id and end_station_id columns in the query. Instead we can just use the station names.

This way we scan less data.

#### **Objective 3. -** 
First of all we find that in which major cities does the company 'Austin Bikes' operates and provide bike trips. Write a SQL query to fetch the same.

**How would you do that?**

In [ ]:
basic_query = """
              SELECT COUNT(DISTINCT landmark) as unique_cities
              FROM `bigquery-public-data.san_francisco.bikeshare_stations`
              """

# Run the query, and return a pandas DataFrame
result = client.query(basic_query).result().to_dataframe()
result.head()

,unique_cities
0,5


In [ ]:
show_time_to_run(basic_query)

Time to run: 0.984 seconds


**How can we achieve this in lesser time?** 

#### **Use APPROX_COUNT_DISTINCT instead of COUNT_DISTINCT() for large datasets.**

In [ ]:
improved_query = """
                 SELECT APPROX_COUNT_DISTINCT(landmark) as unique_cities
                 FROM `bigquery-public-data.san_francisco.bikeshare_stations`
                 """
              
# Run the query, and return a pandas DataFrame
result = client.query(improved_query).result().to_dataframe()
result.head()

,unique_cities
0,5


In [ ]:
show_time_to_run(improved_query)

Time to run: 0.717 seconds


**Why does this happens?**

The problem with COUNT_DISTINCT is that counting distinct values requires more and more memory as the number of distinct values increases.

The implementation of APPROX_COUNT_DISTINCT() has a much smaller memory requirement as compared to the COUNT(DISTINCT) function. 

APPROX_COUNT_DISTINCT() function is designed to provide aggregations across large data sets where responsiveness is more critical than absolute precision

#### **Objective 4. -** 
The team wants to see if there's any specific pattern in the bike trips from several major cities. Write a SQL query to fetch the details of all trips where the start station is in one of these cities - 'san jose', 'redwood', 'sjsu', 'santa clara', 'san salvador' or 'mezes'. 

**How would you do that?**

In [ ]:
basic_query = """
              SELECT *
              FROM `bigquery-public-data.san_francisco.bikeshare_trips`
              WHERE 
                LOWER(start_station_name) LIKE '%san jose%' OR
                LOWER(start_station_name) LIKE '%redwood%' OR
                LOWER(start_station_name) LIKE '%sjsu%' OR
                LOWER(start_station_name) LIKE '%santa clara%' OR
                LOWER(start_station_name) LIKE '%san salvador%' OR
                LOWER(start_station_name) LIKE '%mezes%'
              """

# Run the query, and return a pandas DataFrame
result = client.query(basic_query).result().to_dataframe()
result.head()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


**How can we improve the readability and conciseness of this query?**

#### **Use 'regexp_contains' to replace multiple 'LIKE' clauses.**

In [ ]:
improved_query = """
                 SELECT *
                 FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                 WHERE
                  REGEXP_CONTAINS(lower(start_station_name),  
                  'san jose|redwood|sjsu|santa clara|santa salvador|mezes')
                 """

# Run the query, and return a pandas DataFrame
result = client.query(improved_query).result().to_dataframe()
result.head()

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


**Why do we do this?**

Using 'regexp_contains' we can list multiple conditions in a single line which looks much clean & concise as compared to the 'LIKE' operator.

#### **Objective 5. -** 
To have some clarity regarding the cities where most of the trips started, write a SQL query to create a seperate column based on, from which of these cities the trip began - 'san jose', 'redwood', 'sjsu', 'santa clara', 'san salvador' or 'mezes'. 

**How would you do that?**

In [ ]:
basic_query = """
              SELECT
              CASE 
                WHEN LOWER(start_station_name) LIKE '%san jose%' THEN 'san jose'
                WHEN LOWER(start_station_name) LIKE '%redwood%' THEN 'redwood'
                WHEN LOWER(start_station_name) LIKE '%sjsu%' THEN 'sjsu'
                WHEN LOWER(start_station_name) LIKE '%santa clara%' THEN 'santa clara'
                WHEN LOWER(start_station_name) LIKE '%san salvador%' THEN 'san salvadore'
                WHEN LOWER(start_station_name) LIKE '%mezes%' THEN 'mezes'
              ELSE 'Other' 
              END AS City
                FROM `bigquery-public-data.san_francisco.bikeshare_trips`
              """

# Run the query, and return a pandas DataFrame
result = client.query(basic_query).result().to_dataframe()
result.head()

,City
0,san jose
1,san jose
2,san jose
3,san jose
4,san jose


In [ ]:
show_time_to_run(basic_query)

Time to run: 2.443 seconds


**How can we achieve this in lesser time?**

#### **Use 'regexp_extract' to replace 'Case-when Like'.**

In [ ]:
improved_query = """
                 SELECT
                  REGEXP_EXTRACT(LOWER(start_station_name), '(san jose|redwood|sjsu|santa clara|santa salvador|mezes)')
                 AS region
                 FROM 
                  `bigquery-public-data.san_francisco.bikeshare_trips`
                 """

# Run the query, and return a pandas DataFrame
result = client.query(improved_query).result().to_dataframe()
result.head()

,region
0,san jose
1,san jose
2,san jose
3,san jose
4,san jose


In [ ]:
show_time_to_run(improved_query)

Time to run: 2.114 seconds


**Why does this happens?**

Adding new search terms in a query can increase the time of execution considerably. 

Also, LOWER() and UPPER() operations have a hard time when dealing with Unicode text. Each character needs to be mapped individually and they can also be multi-byte.

That's why BigQuery case insensitive text search is slower.

Whereas, using 'regexp' can replicate the in lesser time.


#### **Objective 6. -** 
San Francisco is a big city and usually has a good number of bike trips. Find the number of bikes and docks currently available at all stations in San Francisco so that proper restocking can be done. 

**How would you do that?**

In [ ]:
basic_query = """
              SELECT t1.station_id, t1.name,
                t2.bikes_available, t2.docks_available 
              FROM 
                `bigquery-public-data.san_francisco.bikeshare_stations` t1
              JOIN
                `bigquery-public-data.san_francisco.bikeshare_status` t2
              ON t1.station_id = t2.station_id
              WHERE
                t1.landmark = 'San Francisco'
              """

show_time_to_run(basic_query)

Time to run: 13.55 seconds


**How can we achieve this in lesser time?**

#### **Always order your JOINs from larger tables to smaller tables.**

Here 'bikeshare_status' is the larger table and 'bikeshare_stations' is the smaller one.

In [ ]:
improved_query = """
                 SELECT t2.station_id, t2.name,
                  t1.bikes_available, t1.docks_available 
                 FROM 
                   `bigquery-public-data.san_francisco.bikeshare_status` t1
                 JOIN
                   `bigquery-public-data.san_francisco.bikeshare_stations` t2
                 ON t1.station_id = t2.station_id
                 WHERE
                  t2.landmark = 'San Francisco'
                 """

show_time_to_run(improved_query)

Time to run: 9.793 seconds


**Why does this happens?**

Table join order matters for reducing the number of rows that the rest of the query needs to process.

In general practice, keeping the larger table in that initial query may boost the query performance.



#### **Objective 7. -** 
Just like San Francisco, Redwood is also a major city that generates a good number of bike trips. Find the total number of bikes and docks available at each station in Redwood city to estimate the cost of restocking.

**How would you do that?**

In [ ]:
basic_query = """
              SELECT
                station_id,
                SUM(bikes_available) AS no_of_bikes,
                SUM(docks_available) AS no_of_docks
              FROM
                `bigquery-public-data.san_francisco.bikeshare_status`
              WHERE
                station_id IN (
                SELECT
                  DISTINCT(station_id)
                FROM
                  `bigquery-public-data.san_francisco.bikeshare_stations`
                WHERE
                  landmark = 'Redwood City')
                GROUP BY station_id
              """

# Run the query, and return a pandas DataFrame
result = client.query(basic_query).result().to_dataframe()
result.head()

,station_id,no_of_bikes,no_of_docks
0,23,10980586,12433683
1,21,9465510,12701562
2,26,10936531,11997239
3,83,7840285,10600310
4,22,17353868,22441738


In [ ]:
show_time_to_run(basic_query)

Time to run: 0.997 seconds


**How can we achieve this in lesser time with better readability?**

- #### **Avoid subqueries in WHERE clause.**
- #### **Use WITH statements vs. Nested subqueries.**

In [ ]:
improved_query = """
                 WITH t2 
                 AS
                 (SELECT
                   DISTINCT(station_id)
                 FROM
                   `bigquery-public-data.san_francisco.bikeshare_stations`
                 WHERE
                   landmark = 'Redwood City'
                   )
                 SELECT
                   t1.station_id,
                   SUM(bikes_available) AS no_of_bikes,
                   SUM(docks_available) AS no_of_docks
                 FROM
                   `bigquery-public-data.san_francisco.bikeshare_status` AS t1
                 JOIN
                   t2
                 ON t1.station_id = t2.station_id
                 GROUP BY
                   t1.station_id
                 """

# Run the query, and return a pandas DataFrame
result = client.query(improved_query).result().to_dataframe()
result.head()

,station_id,no_of_bikes,no_of_docks
0,25,10208293,12854208
1,83,7840285,10600310
2,23,10980586,12433683
3,21,9465510,12701562
4,26,10936531,11997239


In [ ]:
show_time_to_run(improved_query)

Time to run: 1.243 seconds


**Why do we do this?**

Using CTEs instead of Subqueries improves the readability of a query and is easier to make sense of.

On the other hand, using Subqueries is way more complex and hard to comprehend.

It is not recommended to use Subqueries in WHERE clause because when SQL executes a statement that contains a subquery in the WHERE clause, it will execute the subquery once for every row returned by the main or "outer" SQL statement.


#### **Objective 8. -** 
The team has a general assumption that the older the establishmnet the more popular it'll be. To verify the same assumption, fetch the station ids and their respective date of installation in order starting from the one installed most recently.

**How would you do that?**

In [ ]:
basic_query = """
              SELECT 
                t.station_id, t.installation_date 
              FROM ( 
                SELECT station_id, installation_date,
                RANK() OVER(PARTITION BY station_id ORDER BY installation_date DESC) AS rnk
                FROM `bigquery-public-data.san_francisco.bikeshare_stations`) t
              WHERE rnk = 1
              ORDER BY t.installation_date DESC
              """

# Run the query, and return a pandas DataFrame
result = client.query(basic_query).result().to_dataframe()
result.head()

,station_id,installation_date
0,91,2016-08-04
1,90,2016-08-04
2,89,2016-06-05
3,88,2016-06-05
4,73,2016-05-19


In [ ]:
show_time_to_run(basic_query)

Time to run: 1.012 seconds


**How can we achieve this in lesser time?**

#### **Use MAX() instead of RANK().**

In [ ]:
improved_query = """
                 SELECT station_id,
                   MAX(installation_date) AS doi
                 FROM `bigquery-public-data.san_francisco.bikeshare_stations`
                 GROUP BY 1
                 ORDER BY doi DESC
                 """

# Run the query, and return a pandas DataFrame
result = client.query(improved_query).result().to_dataframe()
result.head()

,station_id,doi
0,90,2016-08-04
1,91,2016-08-04
2,89,2016-06-05
3,88,2016-06-05
4,73,2016-05-19


In [ ]:
show_time_to_run(improved_query)

Time to run: 0.935 seconds


**Why does this happens?**

While using RANK(), first you need to pull all results, then rank them based on your partitioning & ordering and at the end specify a condition to fetch only the rank 1 record. 

Whereas for MAX() you can just group by and fetch the record with maximum date i.e. the most recent from each group.

Also, using MAX() instead of RANK() function definitely seems simpler and more understandable.



#### **Some Other Tips -**

* Avoid N:N joins.
* Use simple equi-joins.
* Avoid UNIONs where possible.
* Use approx_percentile(metric, 0.5) for median.
* Convert long list of IN clauses into a temporary table.

#### **References -** 
- Advanced SQL by Alexis Cook

#### <center>End of Script-3</center>